# Make animations of what component variation looks like

In [1]:
import gzbuilder_analysis.rendering as rg
from gzbuilder_analysis.rendering.sersic import sersic2d
from gzbuilder_analysis.aggregation.spirals import xy_from_r_theta
from gzbuilder_analysis.config import DEFAULT_SPIRAL, DEFAULT_DISK
from asinh_cmap import asinh_cmap
import os
from scipy.optimize import minimize
from tqdm import tqdm

Sersic Axis ratio, $q$

In [2]:
os.makedirs('animations/q', exist_ok=True)

cx, cy = np.mgrid[:250, :250]
!mkdir animations
for i, q in enumerate(np.linspace(0.3, 1, 50)):
    im = sersic2d(cx, cy, 250/2, 250/2, 0, 50, q, 2, 1, 1)
    plt.imshow(im, cmap=asinh_cmap)
    plt.axis('off')
    plt.title(f'$q={q:.2f}$')
    plt.savefig(f'animations/q/{i:03d}.png', bbox_inches='tight')
    plt.close()

mkdir: animations: File exists


And convert these images into a gif:

In [3]:
!convert animations/q/*.png animations/q/one_direction.gif
!convert animations/q/one_direction.gif -coalesce -duplicate 1,-2-1 -quiet -layers OptimizePlus -loop 0 animations/q.gif
!rm animations/q/one_direction.gif

## Sersic n
We attempt to optimize scale and brightness to keep models as similar as possible to an exponential, otherwise models are difficult to compare

In [4]:
os.makedirs('animations/n', exist_ok=True)

I, Re = (1, 50)
im_target = sersic2d(cx, cy, 250/2, 250/2, 0, Re, 1, 2, I, 1)
def make_f(n):
    def f(p):
        I, Re = p
        y = sersic2d(cx, cy, 250/2, 250/2, 0, Re, 1, 2, I, n)
        return ((im_target - y)**2).sum() / im_target.size
    return f

cx, cy = np.mgrid[:250, :250]
!mkdir animations
with tqdm(np.linspace(0.4, 5, 50)) as bar:
    for i, n in enumerate(bar):
        res = minimize(make_f(n), (I, Re))
        I, Re = res['x']
        plt.figure(figsize=(8, 6))
        im = sersic2d(cx, cy, 250/2, 250/2, 0, Re, 1, 2, I, n)
        plt.imshow(im, cmap=asinh_cmap)
        c = plt.colorbar(shrink=0.95)
        c.ax.set_yticklabels([f'{str(i):<3s}' for i in c.ax.get_yticks()])
        plt.axis('off')
        plt.title(f'$n={n:.2f};\;I={I:.2e};\;R_e={Re:.2e}$')
        plt.savefig(f'animations/n/{i:03d}.png', bbox_inches='tight')
        plt.close()

mkdir: animations: File exists


100%|██████████| 50/50 [00:54<00:00,  1.41s/it]


In [5]:
!convert animations/n/*.png animations/n/one_direction.gif
!convert animations/n/one_direction.gif -coalesce -duplicate 1,-2-1 -quiet -layers OptimizePlus -loop 0 animations/n.gif
!rm animations/n/one_direction.gif 

Sersic Boxyness, $c$

In [6]:
os.makedirs('animations/c', exist_ok=True)

I, Re = (1, 50)
im_target = sersic2d(cx, cy, 250/2, 250/2, 0, Re, 1, 2, I, 1)
def make_f(n):
    def f(p):
        I, Re = p
        y = sersic2d(cx, cy, 250/2, 250/2, 0, Re, 1, 2, I, n)
        return ((im_target - y)**2).sum() / im_target.size
    return f

cx, cy = np.mgrid[:250, :250]
!mkdir animations
with tqdm(np.linspace(0.5, 3, 50)) as bar:
    for i, c in enumerate(bar):
        plt.figure(figsize=(8, 6))
        im = sersic2d(cx, cy, 250/2, 250/2, 0, 50, 0.6, c, 1, 1)
        plt.imshow(im, cmap=asinh_cmap)
        plt.axis('off')
        plt.title(f'$c={c:.2f}$')
        plt.savefig(f'animations/c/{i:03d}.png', bbox_inches='tight')
        plt.close()

mkdir: animations: File exists


100%|██████████| 50/50 [00:12<00:00,  4.04it/s]


In [7]:
!convert animations/c/*.png animations/c/one_direction.gif
!convert animations/c/one_direction.gif -coalesce -duplicate 1,-2-1 -quiet -layers OptimizePlus -loop 0 animations/c.gif
!rm animations/c/one_direction.gif 

## Spiral intensity relative to disk

In [8]:
help(rg.calculate_model)

Help on function calculate_model in module gzbuilder_analysis.rendering:

calculate_model(model, image_size=(256, 256), psf=None, oversample_n=5)
    Render a model and convolve it with a psf (if provided)



In [9]:
os.makedirs('animations/spiral', exist_ok=True)

def log_spiral(t, phi, dt=0):
    r = np.exp(np.tan(np.deg2rad(phi)) * t)
    return xy_from_r_theta(r, t + dt).T

t = np.linspace(0, np.pi, 50)
phi = 15
arms = [log_spiral(t, phi).T, log_spiral(t, phi, np.pi).T]
plt.plot(*log_spiral(t, phi).T)
plt.plot(*log_spiral(t, phi, np.pi).T)
n_arms = 2
DEFAULT_SPIRAL['spread'] = 1
spiral = [
    [log_spiral(t, phi, 2 * np.pi / n_arms * i) * 10 + 50, DEFAULT_SPIRAL]
    for i in range(n_arms)
]

with tqdm(np.linspace(0, 100, 50)) as bar:
    for i, mux in enumerate(bar):
        model = {
            'disk': {**DEFAULT_DISK, 'I': 0.2, 'Re': 20, 'mux': mux},
            'bulge': None,
            'bar': None,
            'spiral': spiral
        }
        for points, _ in model['spiral']:
            plt.plot(*points.T, 'r:', alpha=1)
        plt.imshow(rg.calculate_model(model, (100, 100)), cmap=asinh_cmap, vmin=0, vmax=1)
        plt.axis('off')
        plt.savefig(f'animations/spiral/{i:03d}.png', bbox_inches='tight')
        plt.close()

100%|██████████| 50/50 [00:14<00:00,  3.83it/s]


In [10]:
!convert animations/spiral/*.png animations/spiral/one_direction.gif
!convert animations/spiral/one_direction.gif -coalesce -duplicate 1,-2-1 -quiet -layers OptimizePlus -loop 0 animations/spiral.gif
!rm animations/spiral/one_direction.gif 